## Modelos IForest Seleccionado

## 1. Importación de librerías

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from sklearn.cluster import KMeans
from random import sample
from sklearn.ensemble import IsolationForest
from sklearn import metrics


#Mostrar todas las columnas de un dataframe
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

## 2. Importación data

In [18]:
#Seleccionar ruta de la data
dataframe = pd.read_csv(r'C:\GIT_Cump\epic\EPIC003\data\modeling\02_preprocessed.csv', encoding='latin')
dataframe.shape

(13516, 13)

In [19]:
dataframe=dataframe.loc[(dataframe['MTO_TRANSF']>10000) & (dataframe['MTO_TRANSF']<500000)]
dataframe.shape

(13516, 13)

## 3. Feature engineering

In [20]:
#Corrijo nombre de variable PERIODO
dataframe.rename(columns = {'ï»¿PERIODO':'PERIODO'}, inplace = True)


#Creo variable FLG_PAIS que une las categorías de riesgo 1 y missing 2 de la variable FLG_PAR
#Inicializo la variable en cero
dataframe['FLG_PAIS']=0
#Creo las condicionales donde dependiendo de la  categoría de la variable inical pinto como 1
dataframe.loc[(dataframe.FLG_PAR==0),'FLG_PAIS' ] = 0
dataframe.loc[(dataframe.FLG_PAR==1), 'FLG_PAIS' ] = 1
dataframe.loc[(dataframe.FLG_PAR==2) ,'FLG_PAIS' ] = 1

## 4. Modelamiento para universo de riesgo

In [21]:
#Importo modelo kmeans final
import pickle 
loaded_KM = pickle.load(open(r'C:\GIT_CUMP\epic\EPIC003\src\03_models\KMeans1_9seg_stairs.model', "rb"))

In [22]:
#Selecciono solo las variables que necesita el modelo
dataset=dataframe[['MTO_TRANSF', 'CTD_OPE', 'FLG_PEP', 'FLG_PROF', 'FLG_PERFIL','CTDEVAL', 'FLG_PAIS']]

#Estandarizo variables
dataset_cluster = MinMaxScaler().fit_transform(dataset)

In [23]:
#Aplico el modelo cargado sobre la data. Columna N_CLuster es la que tiene el resultado del modelo
dataframe['N_CLUSTER']=loaded_KM.predict(dataset_cluster)
dataframe.head()


,Unnamed: 0,PERIODO,CODCLAVECIC,NBRCLIORDENANTE,SEGMENTO,MTO_TRANSF,CTD_OPE,FLG_PEP,FLG_PROF,FLG_PAR,FLG_PERFIL,CTDEVAL,FLG_PAIS,N_CLUSTER
0,0,202206,812,CORTES DE POLAR SANDRA MIRIAN,ENALTA,100000.00,1,0,1,2,0,0,1,1
1,8,202209,1750,CONROY DE MARSANO MARIA TERESA,PRIVADA,17357.00,1,0,0,0,0,15,0,2
2,9,202204,1973,APARCANA HERRERA MARIELA VILMA,CONSUMO,13931.65,1,0,1,0,0,0,0,3
3,12,202204,2249,DANERI PREIS GUSTAVO,ENALTA,13275.00,1,0,1,0,0,5,0,3
4,14,202208,2374,FIEDLER VASQUEZ-MEJIA PAUL,ENALTA,10670.96,2,0,1,0,0,0,0,3


In [24]:
dataframe.shape

(13516, 14)

In [35]:
#Exporto la data con la columna adicional
dataframe.to_csv(r'C:\GIT_CUMP\epic\EPIC003\data\modeling\03_predicted.csv',index=False)

#Código si se necesita separador "|""
#dataset.to_csv(r'Output_model.csv',index=False,sep='|')

In [27]:
#Defino Universo basado en clusters de riesgo seleccionados
dataseg=dataframe[dataframe['N_CLUSTER'].isin([4,5,6,7,8])]
dataseg.shape

(1217, 14)

## 5. Modelamiento para obtención de alertas

In [28]:
#Selecciono las variables para el seundo modelo de IForest
dsrisk=dataseg[['MTO_TRANSF', 'CTD_OPE', 'FLG_PEP', 'FLG_PROF', 'FLG_PERFIL','CTDEVAL', 'FLG_PAIS']]

In [29]:
#Cargo el modelo de IForest
loaded_IF = pickle.load(open(r'C:\GIT_CUMP\epic\EPIC003\src\03_models\IF_EPIC003_alerta.model', "rb"))

In [30]:
#Aplico el modelo cargado sobre la data. Columna Outlier es la que tiene el resultado del modelo

dataseg['OUTLIER']=loaded_IF.predict(dsrisk)

dataseg.head()

,Unnamed: 0,PERIODO,CODCLAVECIC,NBRCLIORDENANTE,SEGMENTO,MTO_TRANSF,CTD_OPE,FLG_PEP,FLG_PROF,FLG_PAR,FLG_PERFIL,CTDEVAL,FLG_PAIS,N_CLUSTER,OUTLIER
8,39,202206,4319,SORIA SUAREZ ALCIDES WILLIAM,ENALTA,100000.00,1,0,0,0,1,0,0,4,1
14,62,202209,6083,CALMET PAPANICOLAU GONZALO,ENALTA,102653.00,3,0,1,0,1,1,0,6,1
15,64,202205,6420,ARAUJO ZAPATA MILAGRITOS FRANCISCA,EXCLUSIVO,20000.00,1,1,0,0,0,0,0,8,1
30,115,202208,9940,LEGUIA OREZZOLI JOAQUIN FELIPE,ENALTA,42096.52,3,0,0,0,1,1,0,4,1
51,184,202204,12810,RUBIN DE-COL JOSE ANTONIO,ENALTA,30000.00,1,0,1,0,1,0,0,6,1


In [31]:
#Valido número de outliers obtenidos
dataseg['OUTLIER'].value_counts()

 1    1095
-1     122
Name: OUTLIER, dtype: int64

In [33]:
#Exporto la data con la columna adicional
dataseg.to_csv(r'C:\GIT_CUMP\epic\EPIC003\data\modeling\04_predicted.csv',index=False)

#Código si se necesita separador "|""
#dataset.to_csv(r'Output_model.csv',index=False,sep='|')

In [34]:
#Pinto version de scikit en el que fue realizado el modelo
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.2.0.
